In [1]:
import platform
print(platform.architecture())

('64bit', 'WindowsPE')


In [2]:
!python --version

Python 3.6.10 :: Anaconda, Inc.


In [3]:
pwd

'C:\\Users\\bae\\셀레니움 크롤러'

In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys
import os
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm
import re

#드라이버 설치
https://chromedriver.chromium.org/downloads
#드라이버랑 크롬버젼이랑 일치시켜주어야함. 드라이버 다운 받을 때 참고(메뉴 => 도움말 => Chrome 정보에서 확인)

In [74]:
#드라이버랑 크롬버젼이랑 일치시켜주어야함. 드라이버 다운 받을 때 참고

#Step 1. 크롬 웹브라우저 실행
#경로설정
path = "C:/Users/bae/Desktop/chromedriver.exe"   # 맥은 "chromedriver"
driver = webdriver.Chrome(path)

#start url 설정. 카테고리별로 다름
driver.get('http://www.tmon.co.kr/deallist/52010000')
time.sleep(2)

# 오늘 하루 보지않기 클릭
try :
    driver.find_element_by_xpath("""/html/body/div[2]/div/button/span""").click( )
except :
    pass

In [75]:
#상품 더 보기가 있다면 상품 더 보기 클릭
try :
    driver.find_element_by_xpath("""//*[@id="_btnMoreDealList"]""").click( )
except :
    pass

In [76]:
SCROLL_PAUSE_TIME = 2

#긁을 카테고리별 시작 url 리스트 ex:과일,채소
start_url_ls = ["http://www.tmon.co.kr/deallist/52030000", "http://www.tmon.co.kr/deallist/52160000"]

for url, category in zip(start_url_ls,['과일', '채소']) :
    driver.get(url +"#sortType=BUY_COUNT")   #판매수 필터로 설정
    time.sleep(2)
    
    #상품 더 보기가 있다면 상품 더 보기 클릭
    try :
        driver.find_element_by_xpath("""//*[@id="_btnMoreDealList"]""").click( )
    except :
        pass

    #scroll 마지막 길이 초기화
    last_height = 0

    #상품 목록 총 개수
    total_count = driver.find_element_by_xpath('//*[@id="_dealSize"]').text
    print("product total count : {} ".format(total_count))

    while True:
        # 화면 최하단으로 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 페이지 로드를 기다림
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 새로운 높이가 이전 높이와 변하지 않았을 경우 스크롤 종료
        if new_height == last_height:
            break
        
        productInfoList = driver.find_elements_by_class_name('item')

        # 스크롤 다운이 된다면 스크롤 다운이 된 후의 창 높이를 새로운 높이로 갱신
        last_height = new_height 
        
        #마지막 제품의 구매횟수가 0이라면 스크롤 종료
        try :
            if productInfoList[-9].find_element_by_class_name('buy_count').text == '' :
                break
        except :
            break

        print("{} scrolling : {}/{}".format(category, len(productInfoList), total_count))
        scroll_count += 200

    #상품 리스트 불러오기    
    productInfoList = driver.find_elements_by_class_name('item') 

    #상품 리스트 텍스트 파일로 저장
    f = open('productUrlList_{}.txt'.format(category), mode='wt', encoding='utf-8')
    for i in range(len(productInfoList)) :
        pp2 = productInfoList[i].find_element_by_tag_name('a')
        f.write(pp2.get_property('href') + '\n')
    f.close()

product total count : 17,323 
과일 scrolling : 408/17,323
과일 scrolling : 608/17,323
과일 scrolling : 808/17,323
과일 scrolling : 1008/17,323
과일 scrolling : 1208/17,323
과일 scrolling : 1408/17,323
과일 scrolling : 1607/17,323
과일 scrolling : 1807/17,323
과일 scrolling : 2007/17,323
과일 scrolling : 2207/17,323
product total count : 13,203 
채소 scrolling : 308/13,203
채소 scrolling : 508/13,203
채소 scrolling : 708/13,203
채소 scrolling : 908/13,203
채소 scrolling : 1108/13,203
채소 scrolling : 1308/13,203
채소 scrolling : 1508/13,203
채소 scrolling : 1708/13,203
채소 scrolling : 1908/13,203
채소 scrolling : 2108/13,203
채소 scrolling : 2308/13,203
채소 scrolling : 2508/13,203
채소 scrolling : 2708/13,203
